In [275]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [276]:
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0'}

In [277]:
response = requests.get(url,headers=headers)

In [278]:
response

<Response [200]>

In [279]:
soup = BeautifulSoup(response.text,'lxml')

In [280]:
titles = soup.find_all("h3",class_='ipc-title__text')

In [281]:
movie_title = []
for title in titles:
    movie_title.append(title.text)

In [282]:
len(movie_title)

263

In [283]:
del movie_title[0]

In [284]:
del movie_title[250:]

In [285]:
name = []
for m in movie_title:
    name.append(m.split('.')[1])

In [286]:
final_name=[]
for n in name:
    final_name.append(n.strip(' '))

In [287]:
final_name[0:5]

['The Shawshank Redemption',
 'The Godfather',
 'The Dark Knight',
 'The Godfather Part II',
 '12 Angry Men']

In [288]:
release_length_rate = soup.find_all('div', class_='sc-b85248f1-5 kZGNjY cli-title-metadata')

In [289]:
year = []
length = []
rating = []

for rlr in release_length_rate:
    elements=rlr.find_all('span', class_='sc-b85248f1-6 bnDqKN cli-title-metadata-item')
    if len(elements) >= 3:
        year.append(elements[0].text)
        length.append(elements[1].text)
        rating.append(elements[2].text)
    else:
        year.append(elements[0].text)
        length.append(elements[1].text)
        rating.append('NA')
    

In [290]:
length[0:5]

['2h 22m', '2h 55m', '2h 32m', '3h 22m', '1h 36m']

In [291]:
rating[0:5]

['R', 'R', 'PG-13', 'R', 'Approved']

In [292]:
year[76]

'1981'

In [293]:
stars_reviews = soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')

In [294]:
stars = []
reviews = []
for sr in stars_reviews:
    stars.append(sr.text.split('(')[0])
    reviews.append(sr.text.split('(')[1].replace(')',''))

In [295]:
stars[:5]

['9.3\xa0', '9.2\xa0', '9.0\xa0', '9.0\xa0', '9.0\xa0']

In [296]:
star = []
for _ in stars:
    star.append(_[:3])

In [297]:
star[:5]

['9.3', '9.2', '9.0', '9.0', '9.0']

In [298]:
reviews[:5]

['2.8M', '1.9M', '2.8M', '1.3M', '829K']

In [299]:
imdb = {
    'Title' : final_name,
    'Year' : year,
    'Length' : length,
    'Rating' : rating,
    'Stars' : star,
    'Reviews':reviews
}

In [300]:
df = pd.DataFrame(imdb)

In [301]:
df.loc[[76]]

,Title,Year,Length,Rating,Stars,Reviews
76,Das Boot,1981,2h 29m,NA,8.4,258K


In [302]:
def con(row):
    
    minutes=df['Length'][row].split(' ')
    if len(minutes)>1:
        l_min=float(minutes[0][:-1])*60+float(minutes[1][:-1])
    else:
        l_min=float(minutes[0][:-1])*60
    return l_min    

In [303]:
movie_length = []
for _ in range(250):
    movie_length.append(con(_))

In [304]:
df['Duration(m)']=movie_length

In [305]:
df.drop(['Length'],axis=1,inplace=True)

In [306]:
df

,Title,Year,Rating,Stars,Reviews,Duration(m)
0,The Shawshank Redemption,1994,R,9.3,2.8M,142.0
1,The Godfather,1972,R,9.2,1.9M,175.0
2,The Dark Knight,2008,PG-13,9.0,2.8M,152.0
3,The Godfather Part II,1974,R,9.0,1.3M,202.0
4,12 Angry Men,1957,Approved,9.0,829K,96.0
...,...,...,...,...,...,...
245,The Help,2011,PG-13,8.1,478K,146.0
246,Persona,1966,Not Rated,8.1,126K,85.0
247,Life of Brian,1979,R,8.0,414K,94.0
248,Aladdin,1992,G,8.0,447K,90.0


In [307]:
df['Rating'].unique()

array(['R', 'PG-13', 'Approved', 'PG', '18+', 'Not Rated', 'G', 'Passed',
       'NA', 'TV-PG', 'Unrated', 'X', '16+', 'TV-MA', 'GP'], dtype=object)

In [308]:
df['Rating'].nunique()

15

In [309]:
df.groupby(['Rating']).count().sort_values(by='Title',ascending=False)

,Title,Year,Stars,Reviews,Duration(m)
Rating,,,,,
R,97,97,97,97,97
PG,37,37,37,37,37
PG-13,34,34,34,34,34
Not Rated,25,25,25,25,25
G,18,18,18,18,18
Passed,16,16,16,16,16
Approved,14,14,14,14,14
18+,2,2,2,2,2
16+,1,1,1,1,1


In [310]:
df['Rating'].replace(['NA','Unrated'],'Not Rated',inplace=True)
df['Rating'].replace(['Passed','Approved'],'G',inplace=True)
df['Rating'].replace(['GP','TV-PG'],'PG',inplace=True)
df['Rating'].replace(['X','TV-MA','18+','16+'],'R',inplace=True)

In [311]:
df['Rating'].unique()

array(['R', 'PG-13', 'G', 'PG', 'Not Rated'], dtype=object)

In [312]:
df.groupby(['Rating']).count().sort_values(by='Title',ascending=False)

,Title,Year,Stars,Reviews,Duration(m)
Rating,,,,,
R,102,102,102,102,102
G,48,48,48,48,48
PG,39,39,39,39,39
PG-13,34,34,34,34,34
Not Rated,27,27,27,27,27


In [313]:
df.head()

,Title,Year,Rating,Stars,Reviews,Duration(m)
0,The Shawshank Redemption,1994,R,9.3,2.8M,142.0
1,The Godfather,1972,R,9.2,1.9M,175.0
2,The Dark Knight,2008,PG-13,9.0,2.8M,152.0
3,The Godfather Part II,1974,R,9.0,1.3M,202.0
4,12 Angry Men,1957,G,9.0,829K,96.0


In [327]:
Review = []
for _ in range(len(df)):
    if df['Reviews'][_][-1]=='M':
        Review.append(float(df['Reviews'][_][:-1])*1000000)
    elif df['Reviews'][_][-1]=='K':
        Review.append(float(df['Reviews'][_][:-1])*1000)
    else:
        Review.append(float(df['Reviews'][_][:-1]))

In [329]:
df['Review']=Review

In [331]:
df.drop(['Reviews'],axis=1,inplace=True)

In [332]:
df.head()

,Title,Year,Rating,Stars,Duration(m),Review
0,The Shawshank Redemption,1994,R,9.3,142.0,2800000.0
1,The Godfather,1972,R,9.2,175.0,1900000.0
2,The Dark Knight,2008,PG-13,9.0,152.0,2800000.0
3,The Godfather Part II,1974,R,9.0,202.0,1300000.0
4,12 Angry Men,1957,G,9.0,96.0,829000.0
